<a href="https://colab.research.google.com/github/anothermartz/Easy-Wav2Lip/blob/v8.3/Easy_Wav2Lip_v8.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to my Easy Wav2Lip colab!

My goal is to make lipsyncing with this tool easy, fast and great looking!

Please view the GitHub for instructions: [https://github.com/anothermartz/Easy-Wav2Lip](https://github.com/anothermartz/Easy-Wav2Lip?tab=readme-ov-file#best-practices)

In [ ]:
version = 'v8.3'
#@title <h1>Step 1: Setup "Easy-Wav2Lip"</h1> With one button: it's really that easy!
#@markdown 👈 Click that little circle play button first - it will ask for Google Drive access: <br>
#@markdown > Accept if your files are on Google Drive (recommended).
#@markdown <br> Alternatively, you can click deny and upload files manually, but this is slower.

#check if already installed
import os
import sys
if os.path.exists('installed.txt'):
    with open('last_file.txt', 'r') as file:
        last_file = file.readline()
    if last_file == version:
        sys.exit('Easy-Wav2Lip '+version+' has already been run on this instance!')


#check GPU is enabled
print('checking for GPU')
import torch
if not torch.cuda.is_available():
    sys.exit('No GPU in runtime. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')

#prompt to mount google drive
print('requesting Google Drive access')
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("google drive not linked")


#start timer
import time
start_time = time.time()

# #clone git
# giturl = 'https://github.com/anothermartz/Easy-Wav2Lip.git'


# !git clone -b {version} {giturl}
# %cd 'Easy-Wav2Lip'
working_directory = os.getcwd()
!mkdir 'face_alignment' 'temp'

#install prerequisites
print('installing batch_face')
import warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        module='torchvision.transforms.functional_tensor')
!pip install batch_face --quiet
!pip install basicsr==1.4.2 --quiet
print('fixing basicsr degradations.py')
# !cp /content/Easy-Wav2Lip/degradations.py /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py
!cp degradations.py /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/basicsr/data/degradations.py
print('installing gfpgan')
!pip install gfpgan --quiet

!python install.py

from IPython.display import clear_output
clear_output()
print("Installation complete, move to Step 2!")

#end timer
elapsed_time = time.time() - start_time
from easy_functions import format_time
print(f"Execution time: {format_time(elapsed_time)}")

In [ ]:
if not os.path.exists('installed.txt'):
    sys.exit('Step 1 has not been run in this instance! Please run step 1 each time you disconnect from a runtime.')
time
############################## user inputs #####################################
#@markdown <h1>Step 2: Select inputs:</h1>

# @markdown On destktop: <h1></h1>Click the folder icon ( 📁 ) at the left edge of colab, find your file, right click, copy path, paste it below:
#@markdown<br></br>
# @markdown On mobile: <h1></h1>Tap the hamburger button ( ☰ ) at the top left, click show file browser, find your file, long press on it, copy path, paste below:
video_file = "../inputs/train_close_mouth_480p.mp4" #@param {type:"string"}
vocal_file = "../inputs/test.wav" #@param {type:"string"}

#@markdown > Keep vocal_file blank if your video already has the desired speech audio encoded into it.
#@markdown # Quality
quality = "Improved" # @param ["Fast", "Improved", "Enhanced"]
#@markdown * <b><u>Fast</u></b>: Wav2Lip <br>
#@markdown * <b><u>Improved</u></b>: Wav2Lip with a feathered mask around the mouth to remove the square around the face <br>
#@markdown * <b><u>Enhanced</u></b>: Wav2Lip + mask + GFPGAN upscaling done on the face
#preview_quality = False #@param {type:"boolean"} - coming soon!
output_height = "full resolution" #@param ["half resolution", "full resolution", "480"] {allow-input: true}
use_previous_tracking_data = True #@param {type:"boolean"}
#@markdown Speeds up processing of the same video used multiple times - it should delete the last tracking file automatically when the video is changed but if it's failing after the first video, untick this box.

#@markdown
#------------------------------*Step 3*----------------------------------------!
#@markdown <h1>👈 Step 3:  Click the little circle play button on this cell! </h1> (Or press ctrl + F10) - Then wait for processing to complete.
# scale padding with resolution
#@markdown <br>

#@markdown ---
#@markdown <br>

#@markdown # [Advanced tweaking](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#advanced-tweaking) (optional) </h1>Just ignore all of this if you are new, or click the blue titles for instructions.
wav2lip_version = "Wav2Lip_GAN" # @param ["Wav2Lip", "Wav2Lip_GAN"]
nosmooth = True #@param {type:"boolean"}
#@markdown ### [Padding:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#padding)</h1> (Up, Down, Left, Right) <br>
U = 0 #@param {type:"slider", min:-100, max:100, step:1}
D = 10 #@param {type:"slider", min:-100, max:100, step:1}
L = 0 #@param {type:"slider", min:-100, max:100, step:1}
R = 0 #@param {type:"slider", min:-100, max:100, step:1}
#@markdown <br>

#@markdown ### [Mask:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)
size = 1.5 #@param {type:"slider", min:1, max:6, step:0.1}
feathering = 1 #@param {type:"slider", min:0, max:3, step:1}
mouth_tracking = False #@param {type:"boolean"}
debug_mask = False #@param {type:"boolean"}


#@markdown # [Other options:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)

batch_process = False #@param {type:"boolean"}
output_suffix = "_Easy-Wav2Lip" #@param {type:"string"}
include_settings_in_suffix = False #@param {type:"boolean"}
preview_input = False #@param {type:"boolean"}
preview_settings = False #@param {type:"boolean"}
#@markdown preview_settings processes only one frame so you can see how it looks without doing the whole video
frame_to_preview = 100 # @param {type:"integer"}
preview_window = 'Both'
wav2lip_batch_size = 32 # @param {type:"integer"}


import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Put all your variables in a dictionary
options = {
    'video_file': video_file,
    'vocal_file': vocal_file,
    'quality': quality,
    'output_height': output_height,
    'wav2lip_version': wav2lip_version,
    'use_previous_tracking_data': use_previous_tracking_data,
    'nosmooth': nosmooth,
    'preview_window': preview_window,
    'wav2lip_batch_size': wav2lip_batch_size
}
padding = {
    'U': U,
    'D': D,
    'L': L,
    'R': R
}
mask = {
    'size': size,
    'feathering': feathering,
    'mouth_tracking': mouth_tracking,
    'debug_mask': debug_mask
}
other = {
    'batch_process': batch_process,
    'output_suffix': output_suffix,
    'include_settings_in_suffix': include_settings_in_suffix,
    'preview_input': preview_input,
    'preview_settings': preview_settings,
    'frame_to_preview': frame_to_preview
}


# Add the dictionary to the ConfigParser object
config['OPTIONS'] = options
config['PADDING'] = padding
config['MASK'] = mask
config['OTHER'] = other

# Write the data to an INI file
with open('config.ini', 'w') as f:
    config.write(f)

!python run.py

# from easy_functions import show_video
# from IPython.display import Image
# if preview_settings:
#     if os.path.isfile(os.path.join('temp','preview.jpg')):
#         display(Image(os.path.join('temp','preview.jpg')))
# else:
#     if os.path.isfile(os.path.join('temp','output.mp4')):
#         print(f"Loading video preview...")
#         show_video(os.path.join('temp','output.mp4'))